# Aprendizado de Máquina

### Teste paramétricos para elaboração de Stacking de modelo de Gradient Boosting com Rede Neural. Ainda em fases de teste

In [55]:
import plotly.express as px
import pandas as pd
import numpy as np
import requests
import gzip

In [56]:
from statsmodels.tsa.filters.hp_filter import hpfilter
from yaml import safe_load
import torch

In [57]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

In [203]:
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from sklearn.preprocessing import MinMaxScaler
from pytorch_forecasting.metrics import QuantileLoss
from sktime.performance_metrics.forecasting import smape_loss

In [59]:
SEED = 4

In [60]:
url = 'https://github.com/wcota/covid19br/blob/master/cases-brazil-cities-time.csv.gz?raw=true'
r = requests.get(url, allow_redirects=True)
open('data.csv.gz','wb').write(r.content)
gz = gzip.open('data.csv.gz')
df = pd.read_csv(gz)

In [61]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [62]:
with open('config.yml') as f:
    config = safe_load(f)

## Validação de Modelos

In [63]:
def to_zero(x):
    if x < 0:
        return 0
    return x

In [64]:
data = df.query("state == 'AM'").groupby('date').sum()
data.index = pd.to_datetime(data.index)
data.index.freq = 'D'
data.drop(columns=['epi_week', 'ibgeID', 'cod_RegiaoDeSaude', 'deaths', 'totalCases'], inplace=True)
data = data.iloc[:,:2]

In [65]:
data['newDeaths'] = data['newDeaths'].apply(to_zero)
data['newCases'] = data['newCases'].apply(to_zero)

In [66]:
for col in data.columns:
    data['log' + "_" + col] = np.log(data[col] + 1)

In [67]:
for col in data.columns[:2]:
    cycle, trend = hpfilter(data[col])
    data[col + '_' + 'cycle'] = cycle.round()
    data[col + '_' + 'trend'] = trend.round()

In [68]:
data['newDeaths_trend'] = data['newDeaths_trend'].apply(to_zero)
data['newCases_trend'] = data['newCases_trend'].apply(to_zero)
data['dia_da_semana'] = data.index.day_name()
data.reset_index(inplace=True)
data['date'] = pd.to_datetime(data['date'])
data_from_newCases = data[['date', 'newCases', 'log_newCases', 'newCases_trend','newCases_cycle', 'dia_da_semana']]
data_from_newCases['MA_7'] = data_from_newCases['newCases'].rolling(7).mean()
data_from_newCases = data_from_newCases.fillna(0)

<ipython-input-68-361b6790fdb9>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [69]:
data_from_newCases['group_ids'] = 1

In [70]:
scaler = MinMaxScaler([0,10])

In [71]:
data_from_newCases['newCases_cycle'] = scaler.fit_transform(data_from_newCases[['newCases_cycle']])
data_from_newCases['MA_7'] = scaler.fit_transform(data_from_newCases[['MA_7']])

In [72]:
data_from_newCases["time_idx"] = range(len(data_from_newCases['date'])) #time_idx
training_cutoff = data_from_newCases["time_idx"].max() - config['max_prediction_length']['value']

In [73]:
def special_days(x):
    if x == 'Monday' or x == 'Sunday':
        return '1'
    return '0'

In [74]:
data_from_newCases['dia_da_semana'] = data_from_newCases['dia_da_semana'].apply(special_days)
data_from_newCases.rename(columns={'dia_da_semana': 'is_special_day'}, inplace=True)

In [75]:
training = TimeSeriesDataSet(
    data_from_newCases[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=config['target']['value'],
    group_ids=['group_ids'],
    min_encoder_length= config['max_encoder_length']['value'] // 2,
    max_encoder_length= config['max_encoder_length']['value'],
    time_varying_known_categoricals=['is_special_day'],
    max_prediction_length=config['max_prediction_length']['value'],
    min_prediction_length=1,
    time_varying_unknown_reals=config['time_varying_unknown_reals']['value'],
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    )

In [76]:
validation = TimeSeriesDataSet.from_dataset(training, data_from_newCases, predict=True, stop_randomization=True)
batch_size = 128  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers = 0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers = 0)

In [77]:
actuals = list(val_dataloader)[0][0]['encoder_target'][0][-14:]
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

72.21428680419922

In [78]:
pl.seed_everything(42)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10,      verbose=False, mode="min")

trainer = pl.Trainer(
    gpus= 1,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=config['gradient_clip_val']['value'],
    callbacks=[early_stop_callback]
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=config['learning_rate']['value'],
    hidden_size=config['hidden_size']['value'],  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=config['attention_head_size']['value'],
    dropout=config['dropout']['value'],  # between 0.1 and 0.3 are good values
    hidden_continuous_size=config['hidden_continuous_size']['value'],  # set to <= hidden_size
    output_size= config['output_size']['value'],  # 7 quantiles by default
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=config['reduce_on_plateau_patience']['value'],
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Number of parameters in network: 8.8k


In [79]:
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 2     
3  | prescalers                         | ModuleDict                      | 128   
4  | static_variable_selection          | VariableSelectionNetwork        | 1.2 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 2.3 K 
6  | decoder_variable_selection         | VariableSelectionNetwork        | 460   
7  | static_context_variable_selection  | GatedResidualNetwork            | 378   
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 378   
9  | static_context_initial_cell_lstm 

1

In [80]:
best_model_path = trainer.checkpoint_callback.best_model_path

In [81]:
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [82]:
actuals = list(val_dataloader)[0][0]['encoder_target'][0][-14:]
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean().item()

34.200687408447266

In [211]:
fig = px.line(y=[predictions[0].round(), actuals, data_from_newCases['newCases'][-14:]], line_shape='spline', labels={'variable': 'Variáveis', 'index': 'Índice', 'value': 'Quantidade'})

customLegend(fig=fig,nameSwap = {'wide_variable_0': 'NN', 'wide_variable_1': 'H&W', 'wide_variable_2': 'Número de Casos'})

In [84]:
from lightgbm import Dataset
from lightgbm import LGBMRegressor

In [146]:
X_data = data_from_newCases.iloc[:,1:].drop(columns=['newCases_trend'])

In [147]:
y_data = data_from_newCases['newCases_trend']

In [148]:
X_train, X_test = X_data[:-14], X_data[-14:]
y_train, y_test = y_data[:-14], y_data[-14:]

In [149]:
model = LGBMRegressor()

In [150]:
X_train['is_special_day'] = data_from_newCases['is_special_day'].apply(lambda x: int(x))

<ipython-input-150-e414b61fae3f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [151]:
X_test['is_special_day'] = data_from_newCases['is_special_day'].apply(lambda x: int(x))

<ipython-input-151-40c97ce9f5e1>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [152]:
model.fit(X_train, y_train)

LGBMRegressor()

In [166]:
y_pred = pd.Series(model.predict(X_test), index=y_test.index)

In [200]:
smape_loss(y_test, y_pred)

0.09063270756759424

In [201]:
smape_loss(y_test, pd.Series(predictions[0], index=y_test.index))

0.23393949841390768

In [202]:
smape_loss(y_test, (predictions[0] + y_pred * 2 ) / 3)

0.13980185112077684

In [191]:
def customLegend(fig, nameSwap):
    for i, dat in enumerate(fig.data):
        for elem in dat:
            if elem == 'name':
                fig.data[i].name = nameSwap[fig.data[i].name]
    return(fig)

#customLegend(fig=fig,nameSwap = {'0': 'Sim', '1': 'Não'})

In [207]:
fig = px.line(y=[y_pred.round(), y_test, data_from_newCases['newCases'][-14:]], line_shape='spline', labels={'variable': 'Variáveis', 'index': 'Índice', 'value': 'Quantidade'})

customLegend(fig=fig,nameSwap = {'wide_variable_0': 'Light GBM', 'wide_variable_1': 'H&W', 'wide_variable_2': 'Número de Casos'})

In [210]:
fig = px.line(y=[((predictions[0] + y_pred*2) / 3).round(), y_test, data_from_newCases['newCases'][-14:]], line_shape='spline', labels={'variable': 'Variáveis', 'index': 'Índice', 'value': 'Quantidade'})

customLegend(fig=fig,nameSwap = {'wide_variable_0': 'Light GBM + NN', 'wide_variable_1': 'H&W', 'wide_variable_2': 'Número de Casos'})